# Experiment 3: Choosing between earlier and later labels, after observing two groups

January 2025

In this experiment, participants observe the gameplay of two groups (the red group and the blue group), and choose between earlier and later labels based on their audience (the red group, the blue group, or either group) and goal (refer to the correct tangram, or be identified as a member of the audience group)

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from memo import memo
import jax
import jax.numpy as jnp
import optax

from enum import IntEnum

import utils

pd.set_option('future.no_silent_downcasting', True)
sns.set_context('poster', font_scale=1.3)

## Model

In [6]:
class TangramTypes(IntEnum):
    shared = 0
    unique = 1


class Responses(IntEnum):
    later = 0
    earlier = 1  # this coding is counterintuitive, but corresponds to utterance length


class AudienceConditions(IntEnum):
    either = 0
    one = 1


class Audiences(IntEnum):
    ingroup = 1
    outgroup = 0

In [7]:
# type: ignore
@jax.jit
def audience_wpp(audience_condition, audience):
    # for the "either group" condition, return 1 regardless of audience
    # for the "one group" condition, return 1 for the ingroup and 0 for the outgroup
    return jnp.array([1, audience])[audience_condition]


@jax.jit
def ref_info(audience, tangram_type, utterance):
    ingroup_info = jnp.array([1, 1])  # [later, earlier]
    outgroup_info = jax.lax.cond(
        tangram_type == TangramTypes.shared,
        lambda _: jnp.array(
            [1, 1]
        ),  # if it's a shared-label tangram, the later utterance is informative regardless of group
        lambda _: jnp.array([0, 1]),
        operand=None,
    )

    info = jax.lax.cond(
        audience == Audiences.ingroup,
        lambda _: ingroup_info,
        lambda _: outgroup_info,
        operand=None,
    )

    return info[utterance]


@jax.jit
def social_info(utterance):
    return jnp.array([1, 0])[utterance]


@jax.jit
def cost(utterance):
    return jnp.array([0, 1])[utterance]


@memo
def speaker[
    utterance: Responses, audience: Audiences
](
    audience_condition: AudienceConditions,
    tangram_type: TangramTypes,
    alpha,
    w_r,
    w_s,
    w_c,
):
    cast: [speaker]
    speaker: chooses(
        audience in Audiences, wpp=audience_wpp(audience_condition, audience)
    )
    speaker: chooses(
        utterance in Responses,
        wpp=exp(
            alpha
            * (
                w_r * ref_info(audience, tangram_type, utterance)
                + w_s * social_info(utterance)
                - w_c * cost(utterance)
            )
        ),
    )
    return Pr[speaker.utterance == utterance]

## Load data

In [8]:
data = pd.read_csv("../data/earlier-later/selection_trials_clean.csv")
data.replace(
    {"response.earlier": {"earlier": Responses.earlier, "later": Responses.later}},
    inplace=True,
)
data.replace(
    {"shared": {"shared": TangramTypes.shared, "unique": TangramTypes.unique}},
    inplace=True,
)  # is it a shared or group-specific label?

data["shared"] = data["shared"].astype(int)
data["response.earlier"] = data["response.earlier"].astype(int)

data.head()

subject_id  item_id counterbalance     condition  \
0  c3340717-2879-4eda-abfe-08cd68995b6d        0              a    one social   
1  c3340717-2879-4eda-abfe-08cd68995b6d        0              a  either refer   
2  c3340717-2879-4eda-abfe-08cd68995b6d        0              a    one social   
3  c3340717-2879-4eda-abfe-08cd68995b6d        0              a     one refer   
4  c3340717-2879-4eda-abfe-08cd68995b6d        0              a     one refer   

     goal audience audience_group tangram  shared  response.earlier  
0  social      one            red       C       0                 0  
1   refer   either           blue       B       0                 0  
2  social      one           blue       L       0                 0  
3   refer      one           blue       L       0                 0  
4   refer      one            red       H       1                 1

Make a jax array for each condition

In [9]:
# masks for each condition
refer_either_mask = data["condition"] == "either refer"
refer_one_mask = data["condition"] == "one refer"
social_one_mask = data["condition"] == "one social"

# get the data for each condition
refer_either_tangram_types = jnp.array(data[refer_either_mask]["shared"].values)
refer_either_responses = jnp.array(data[refer_either_mask]["response.earlier"].values)

refer_one_tangram_types = jnp.array(data[refer_one_mask]["shared"].values)
refer_one_responses = jnp.array(data[refer_one_mask]["response.earlier"].values)

social_one_tangram_types = jnp.array(data[social_one_mask]["shared"].values)
social_one_responses = jnp.array(data[social_one_mask]["response.earlier"].values)

## Predict

Define speaker functions

In [10]:
@jax.jit
def speaker_refer_either(tangram_type, w_r, w_s, w_c):
    return speaker(
        audience_condition=AudienceConditions.either,
        tangram_type=tangram_type,
        alpha=1,
        w_r=w_r,
        w_s=0,
        w_c=w_c,
    )[1, 0]

@jax.jit
def speaker_refer_one(tangram_type, w_r, w_s, w_c):
    return speaker(
        audience_condition=AudienceConditions.one,
        tangram_type=tangram_type,
        alpha=1,
        w_r=w_r,
        w_s=0,
        w_c=w_c,
    )[1, 0]

@jax.jit
def speaker_social_one(tangram_type, w_r, w_s, w_c):
    return speaker(
        audience_condition=AudienceConditions.one,
        tangram_type=tangram_type,
        alpha=1,
        w_r=0,
        w_s=w_s,
        w_c=w_c,
    )[1, 0]

# vmap functions, for vectorizing the functions over the data
speaker_refer_either_vmap = jax.vmap(speaker_refer_either, in_axes=(0, None, None, None))
speaker_refer_one_vmap = jax.vmap(speaker_refer_one, in_axes=(0, None, None, None))
speaker_social_one_vmap = jax.vmap(speaker_social_one, in_axes=(0, None, None, None))

Compute preds

In [11]:
@jax.jit
def predict_all(
    model_type,
    refer_either_tangram_types,
    refer_one_tangram_types,
    social_one_tangram_types,
    w_r,
    w_s,
    w_c,
):
    """Generate predictions for all conditions"""
    preds_refer_either = speaker_refer_either_vmap(
        refer_either_tangram_types, w_r, w_s, w_c
    )
    preds_refer_one = speaker_refer_one_vmap(refer_one_tangram_types, w_r, w_s, w_c)
    preds_social_one = jax.lax.cond(
        model_type == utils.ModelType.base,
        lambda _: speaker_refer_one_vmap(refer_one_tangram_types, w_r, w_s, w_c),
        lambda _: speaker_social_one_vmap(social_one_tangram_types, w_r, w_s, w_c),
        operand=None,
    )
    return preds_refer_either, preds_refer_one, preds_social_one


@jax.jit
def compute_NLL_all(
    model_type,
    refer_either_tangram_types,
    refer_one_tangram_types,
    social_one_tangram_types,
    refer_either_responses,
    refer_one_responses,
    social_one_responses,
    w_r,
    w_s,
    w_c,
):
    """compute NLL of whole dataset"""
    preds_refer_either, preds_refer_one, preds_social_one = predict_all(
        model_type,
        refer_either_tangram_types,
        refer_one_tangram_types,
        social_one_tangram_types,
        w_r,
        w_s,
        w_c,
    )
    NLL_refer_either = utils.compute_NLL(preds_refer_either, refer_either_responses)
    NLL_refer_one = utils.compute_NLL(preds_refer_one, refer_one_responses)
    NLL_social_one = utils.compute_NLL(preds_social_one, social_one_responses)
    return NLL_refer_either + NLL_refer_one + NLL_social_one

Fit params

In [12]:
def fit_params(
    model_type,
    initial_params,
    refer_either_tangram_types,
    refer_one_tangram_types,
    social_one_tangram_types,
    refer_either_responses,
    refer_one_responses,
    social_one_responses,
    lr=0.001,
    tol=1e-6,
    max_steps=10000,
):
    def loss_fn(params):
        return compute_NLL_all(
            model_type,
            refer_either_tangram_types,
            refer_one_tangram_types,
            social_one_tangram_types,
            refer_either_responses,
            refer_one_responses,
            social_one_responses,
            params[0],
            params[1],
            params[2],
        )

    grad_fn = jax.value_and_grad(loss_fn)
    params = jnp.array(initial_params, dtype=jnp.float32)
    optimizer = optax.adam(learning_rate=lr)
    opt_state = optimizer.init(params)

    prev_nll = None
    for step in range(max_steps):
        nll_value, nll_grads = grad_fn(params)
        updates, opt_state = optimizer.update(nll_grads, opt_state)
        params = optax.apply_updates(params, updates)
        # ensure all params are positive
        params = params.at[:].set(jnp.clip(params[:], 0, jnp.inf))

        if step % 1000 == 0:
            print(f"Step {step}, NLL value: {nll_value}, Params: {params}")

        if prev_nll is not None and abs(nll_value - prev_nll) < tol:
            print(f"Converged at step {step} with NLL={nll_value}, Params: {params}")
            break

        prev_nll = nll_value

    best_nll = loss_fn(params)
    return params, best_nll

In [13]:
best_params_base, best_nll_base = fit_params(
    utils.ModelType.base,
    [1, 0, 1],
    refer_either_tangram_types,
    refer_one_tangram_types,
    social_one_tangram_types,
    refer_either_responses,
    refer_one_responses,
    social_one_responses
)

Step 0, NLL value: 1412.2802734375, Params: [1.001 0.    0.999]
Step 1000, NLL value: 1380.82177734375, Params: [1.5889819 0.        0.7142428]
Converged at step 1482 with NLL=1380.26953125, Params: [1.7478418  0.         0.72681576]


In [14]:
best_params_social, best_nll_social = fit_params(
    utils.ModelType.social,
    [1, 1, 1],
    refer_either_tangram_types,
    refer_one_tangram_types,
    social_one_tangram_types,
    refer_either_responses,
    refer_one_responses,
    social_one_responses
)

Step 0, NLL value: 1488.1201171875, Params: [1.001 0.999 0.999]
Step 1000, NLL value: 1378.7725830078125, Params: [1.463796   0.4056054  0.56888974]
Converged at step 1749 with NLL=1377.573486328125, Params: [1.5945642  0.2942899  0.61344683]


Generate model predictions

In [15]:
# Base model preds
preds_refer_either_base, preds_refer_one_base, preds_social_one_base = predict_all(
    utils.ModelType.base,
    refer_either_tangram_types,
    refer_one_tangram_types,
    social_one_tangram_types,
    *best_params_base
)

# Social model preds
preds_refer_either_social, preds_refer_one_social, preds_social_one_social = predict_all(
    utils.ModelType.social,
    refer_either_tangram_types,
    refer_one_tangram_types,
    social_one_tangram_types,
    *best_params_social
)

Put the preds into a dataframe

In [16]:
tangram_types_mapping = lambda x: np.where(x == 0, "shared", "group-specific")
preds_df_base = pd.DataFrame(
    {
        "condition": ["refer_either"] * len(preds_refer_either_base)
        + ["refer_one"] * len(preds_refer_one_base)
        + ["social_one"] * len(preds_social_one_base),
        "goal": ["refer"] * len(preds_refer_either_base)
        + ["refer"] * len(preds_refer_one_base)
        + ["social"] * len(preds_social_one_base),
        "audience": ["either"] * len(preds_refer_either_base)
        + ["one"] * len(preds_refer_one_base)
        + ["one"] * len(preds_social_one_base),
        "tangram_type": tangram_types_mapping(
            np.concatenate(
                [
                    refer_either_tangram_types,
                    refer_one_tangram_types,
                    social_one_tangram_types,
                ]
            )
        ),
        "p_earlier": np.concatenate(
            [
                preds_refer_either_base,
                preds_refer_one_base,
                preds_social_one_base,
            ]
        ),
        "model": "base model",
    }
).drop_duplicates()

preds_df_social = pd.DataFrame(
    {
        "condition": ["refer_either"] * len(preds_refer_either_social)
        + ["refer_one"] * len(preds_refer_one_social)
        + ["social_one"] * len(preds_social_one_social),
        "goal": ["refer"] * len(preds_refer_either_base)
        + ["refer"] * len(preds_refer_one_base)
        + ["social"] * len(preds_social_one_base),
        "audience": ["either"] * len(preds_refer_either_base)
        + ["one"] * len(preds_refer_one_base)
        + ["one"] * len(preds_social_one_base),
        "tangram_type": tangram_types_mapping(
            np.concatenate(
                [
                    refer_either_tangram_types,
                    refer_one_tangram_types,
                    social_one_tangram_types,
                ]
            )
        ),
        "p_earlier": np.concatenate(
            [
                preds_refer_either_social,
                preds_refer_one_social,
                preds_social_one_social,
            ]
        ),
        "model": "social model",
    }
).drop_duplicates()

data_df = pd.DataFrame(
    {
        "condition": ["refer_either"] * len(refer_either_responses)
        + ["refer_one"] * len(refer_one_responses)
        + ["social_one"] * len(social_one_responses),
        "goal": ["refer"] * len(refer_either_responses)
        + ["refer"] * len(refer_one_responses)
        + ["social"] * len(social_one_responses),
        "audience": ["either"] * len(refer_either_responses)
        + ["one"] * len(refer_one_responses)
        + ["one"] * len(social_one_responses),
        "tangram_type": tangram_types_mapping(
            np.concatenate(
                [
                    refer_either_tangram_types,
                    refer_one_tangram_types,
                    social_one_tangram_types,
                ]
            )
        ),
        "p_earlier": np.concatenate(
            [refer_either_responses, refer_one_responses, social_one_responses]
        ),
        "model": "data",
    }
)

preds_df = pd.concat([preds_df_base, preds_df_social, data_df])

In [17]:
preds_df

condition    goal audience    tangram_type  p_earlier       model
0     refer_either   refer   either          shared   0.325894  base model
1     refer_either   refer   either  group-specific   0.530533  base model
720      refer_one   refer      one          shared   0.325894  base model
721      refer_one   refer      one  group-specific   0.325894  base model
1440    social_one  social      one          shared   0.325894  base model
...            ...     ...      ...             ...        ...         ...
2155    social_one  social      one  group-specific   0.000000        data
2156    social_one  social      one          shared   0.000000        data
2157    social_one  social      one          shared   0.000000        data
2158    social_one  social      one          shared   0.000000        data
2159    social_one  social      one          shared   0.000000        data

[2172 rows x 6 columns]

In [18]:
preds_df.to_csv('preds_exp3.csv', index=False)

In [19]:
conditions = [
    (preds_df["goal"] == "refer") & (preds_df["audience"] == "one"),
    (preds_df["goal"] == "refer") & (preds_df["audience"] == "either"),
    (preds_df["goal"] == "social") & (preds_df["audience"] == "one")
]
conditions_labels = ["refer to in-group", "refer to mixed", "social"]
preds_df["condition"] = np.select(conditions, conditions_labels)

In [20]:
g = sns.FacetGrid(
    preds_df,
    row="tangram_type",
    col="model", 
    margin_titles=True,
    height=5.8,
    aspect=0.8
    # legend_out=True
)
g.map_dataframe(
    sns.pointplot,
    x="audience",
    y="p_earlier",
    order=["one", "either"],
    hue="condition",
    hue_order=conditions_labels,
    palette=utils.palette,
    dodge=0.18,
    alpha=1
)
g.set_titles(col_template="{col_name}", row_template="{row_name}")
g.set(ylim=(0, 1), ylabel="$P(\\text{earlier label})$")
for ax in g.axes.flat:
    ax.axhline(0.5, ls="--", color="gray")
g.add_legend(title="condition", bbox_to_anchor=(1.01, 0.5))

plt.savefig(utils.fig_dir + "3pp_red_blue_earlier_later.pdf")
plt.show()

LR test

In [21]:
from scipy.stats import chi2
lr_stat = 2 * (best_nll_base - best_nll_social)
df = 1  # social model has additional param w_s
p_val = chi2.sf(lr_stat, df)
lr_stat, p_val

(Array(5.392334, dtype=float32), 0.020225400724294275)